## (Helpful) References

* http://cs231n.stanford.edu/vecDerivs.pdf
* https://web.stanford.edu/class/cs224n/readings/gradient-notes.pdf

## The derivative of a vector-valued function $\vec h(x,W)$  w.r.t. to some weight-matrix $W$

Suppose the following setup:

<img style="max-width:400px;" src="https://i.imgur.com/sG5UBMq.png"></img>

$\vec h$ is a vector-valued function. We're looking for the derivative $\frac{\partial \vec h}{\partial W}$. This requires us to calculate partial derivatives of *each* element of h with respect to *each* element of W. In total, we have to compute `3*(2*3)=18` partial derivatives. We can store these 18 derivatives in three (2x3) matrices. These matrices will store the partial derivatives of $\frac{\partial h_1}{\partial W}, \frac{\partial h_2}{\partial W}, \frac{\partial h_3}{\partial W}$.

<img style="max-width:500px;" src="https://i.imgur.com/oPLzvtc.png"></img>

When computing the elements of the matrices, we note that the following pattern emerges:

<img style="max-width:500px;" src="https://i.imgur.com/YZ7T3SQ.png"></img>

Using this, we can compute the partial derivatives:

<img style="max-width:500px;" src="https://i.imgur.com/thOmApu.png"></img>

These three matrices can now be used to update `W`. To update `W`, we would need to do three steps:

1. $W = W - \text{learning_rate}\cdot \frac{\partial h_1}{\partial W}$
1. $W = W - \text{learning_rate}\cdot \frac{\partial h_2}{\partial W}$
1. $W = W - \text{learning_rate}\cdot \frac{\partial h_3}{\partial W}$

or, equivalently,

$$
W = W - \text{learning_rate}*(\frac{\partial h_1}{\partial W} + \frac{\partial h_2}{\partial W} + \frac{\partial h_3}{\partial W})
$$

But there is a trick (see Section 3 of http://cs231n.stanford.edu/vecDerivs.pdf) that (1) simplifies the update-step and (2) helps storing the three matrices more efficiently. For this trick, note that most elements of the three matrices are zero. In fact, the only non-zero elements $\frac{\partial h_i}{\partial W_{jk}}$ of $\frac{\partial h_i}{\partial W}$ are those elements where $i=k$. We could define a new matrix $J_{j,i}=\frac{\partial h_i}{\partial W_{ji}}$ that would store the *same* non-trivial information of all three matrices in a *single* 2D-matrix:

<img style="max-width:500px;" src="https://i.imgur.com/BK4NXCP.png"></img>

This matrix is the efficiently stored **Jacobian** $J=\frac{\partial \vec h}{\partial W}$. Using this matrix, the update-step simplifies to:

$$
W = W - \text{learning_rate}\cdot J
$$

---

For example, for input vector $\vec x=[1, 2]$ the efficiently stored Jacobian is:

<img style="max-width:500px;" src="https://i.imgur.com/khVlJIY.png"></img>

In [61]:
# The same example in pytorch
# https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html#optional-reading-tensor-gradients-and-jacobian-products

import torch
x = torch.Tensor([1, 2]).reshape((1, -1))
W = torch.randn((2, 3), requires_grad=True)
h = x@W
print(f"x: {x.shape}\nW: {W.shape}\nh: {h.shape}")

# Normally, we want to compute the gradient of a *scalar* function w.r.t. to some
# parameters. In that case we can simply call f.backward()
# But here, we want to compute the gradient of a *vector-valued* function w.r.t. some
# parameters. In that case f.backward() will compute the *Jacobian product*, and not
# the actual gradient, see https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html#optional-reading-tensor-gradients-and-jacobian-products
v = torch.ones_like(h)
h.backward(v)
J = W.grad # computes v.T@J for input vector v=(v1, ..., vm)
print(f"\n=> Jacobian/Jacobian Product:\n{J}")

x: torch.Size([1, 2])
W: torch.Size([2, 3])
h: torch.Size([1, 3])

=> Jacobian/Jacobian Product:
tensor([[1., 1., 1.],
        [2., 2., 2.]])


**Why is the jacobian product *not* the actual gradient?**

See https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html#optional-reading-tensor-gradients-and-jacobian-products

* "the gradient is subset of the Jacobian." 
* "the gradient can be seen as special case of the Jacobian, i.e. when the function is scalar"
* "The Jacobian matrix is the matrix formed by the partial derivatives of a vector function. Its vectors are the gradients of the respective components of the function." => The Jacobian stores the GRADIENTS of the components of the function in its columns/rows!

## derivatives in a network 

* We extend the example above by adding an output layer associated with weights $\vec w^{(out)}=[w_1^{(out)}, w_2^{(out)}, w_3^{(out)}]$, that takes in $\vec h=[h_1, h_2, h_3]$ and computes $score=\vec w^{(out)}\vec h=w_1^{(out)}h_1 + w_2^{(out)}h_2 + w_3^{(out)}h_3$. 
* Then `score` can then be pushed through a sigmoid function to be turned into a probability: $prob=\sigma(score)$. 
* Then `prob` can be compared with the actual target via some loss function $Loss(prob, target)$.
* Then updating the weights $W$ and $w^{(out)}$ is a matter of finding the derivatives `dLoss/dW` and `dLoss/dwout`.

Here are the steps visualized:

```
x = [x1, x2]     # a single observation has 2 features
h = [h1, h2, h3] # hidden layer with 3 neurons with states h1, h2, h3
```

The weights of each of the three neurons correspond to a column

```
W = [[W11, W12, W13],
     [W21, W22, W23]]
```

Then we can compute the hidden layer

```
h = xW = [h1, h2, h3] # shape(1x3)
```

* x:  shape(1x2)
* W: shape(2x3)
* h: shape(1x3)

Then we run `h` through an output layer with a sigmoid activation function. That output layer has weights `v=[v1, v2, v3]`.

```
out = h@v = [h1 h2 h3] @ [v1 v2 v3] = h1*v1 + h2*v2 + h3*v3
```

To get a probability, we put the result through a sigmoid activation function:

```
prob = sigmoid(out)
```

We can put everything together:


```
f(x, W, v) = sigmoid(out)
           = sigmoid(h @ v)
           = sigmoid(xW @ v)
```

Our goal is now to find `dfdW` and `dfdv` because they are needed to update our weights `W` and `v`.

Using the chain rule, we find `dfdW` as follows:

```
(1)   dprob/dout = sigmoid(out)*sigmoid(1-out)
(2.1) dout/dv    = h
(2.2) dout/dh    = v
(3)   dh/dW = x
```

Applying the chain rule to get `df/dW` and `df/dv`:

```
df/dv = dprob/dout * dout/dv 
      = s(out)*s(1-out) * h

df/dW = dprob/dout * dout/dh * dh/dW
      = s(out)*s(1-out) * v * x
```


Now it's unclear how the vector-vector product `v*x = [v1 v2 v3] * [x1 x2]` is defined? In the end, we must get a (2x3) matrix. The only way to get that from a 3D and 2D vector is via the dot product `shape(2x1) @ shape(1x3) = shape(2x3)`, hence `x.T @ v.T`. But it's unclear why...Maybe it has to do with *Jacobians*?

In [170]:
from torch.nn.functional import relu
from torch import sigmoid

x = torch.Tensor([1, 2])
t = torch.Tensor([1])

W = torch.randn((2, 3), requires_grad=True)
wout = torch.randn(3, requires_grad=True)

# forward
h = x@W
hact = relu(h)
score = hact@wout
prob = sigmoid(score)
loss = (prob - t)**2

print(h.shape, hact.shape, score.shape, prob.shape, loss.shape)
print(loss)

torch.Size([3]) torch.Size([3]) torch.Size([]) torch.Size([]) torch.Size([1])
tensor([0.3632], grad_fn=<PowBackward0>)


In [171]:
# forward with storing gradients
h = x@W
dh_dW = x.repeat(3).reshape((3, 2)).T 

hact = relu(h)
dhact_dh = torch.zeros_like(hact)
dhact_dh[hact > 0] = 1.0

score = hact@wout
dscore_dwout = hact
dscore_dhact = wout

prob = sigmoid(score)
dprob_dscore = sigmoid(score)*(1-sigmoid(score))

loss = (prob - t)**2
dloss_dprob = 2*(prob-t)

In [172]:
# gradients required for updating wout
print(f"dloss/dprob  = {dloss_dprob}")
print(f"dprob/dscore = {dprob_dscore}")
print(f"dscore/dwout = {dscore_dwout}\n")

# applying chain rule
dloss_dwout = dloss_dprob * dprob_dscore * dscore_dwout

# update wout
print(f"wout \t\t= {wout}\ndloss/dwout \t= {dloss_dwout}")
wout_new = wout - 1*dloss_dwout
print(f"wout \t\t= {wout_new}")

dloss/dprob  = tensor([-1.2053], grad_fn=<MulBackward0>)
dprob/dscore = 0.23946520686149597
dscore/dwout = tensor([2.0583, 1.7744, 1.7216], grad_fn=<ReluBackward0>)

wout 		= tensor([-1.3355,  1.2879,  0.0274], requires_grad=True)
dloss/dwout 	= tensor([-0.5941, -0.5121, -0.4969], grad_fn=<MulBackward0>)
wout 		= tensor([-0.7415,  1.8001,  0.5243], grad_fn=<SubBackward0>)


In [173]:
# gradients required for updating W
print(f"dloss/dprob  \t= {dloss_dprob}")
print(f"dprob/dscore \t= {dprob_dscore}")
print(f"dscore/dhact \t= {dscore_dhact}")
print(f"dhact/dh \t= {dhact_dh}")
print(f"dh/dW =\n{dh_dW}\n")

# applying chain rule
dloss_dW = dloss_dprob * dprob_dscore * dscore_dhact * dhact_dh * dh_dW
print(dloss_dW)

# # update W
# print(f"wout \t\t= {wout}\ndloss/dwout \t= {dloss_dwout}")
# wout = wout - 1*dloss_dwout
# print(f"wout \t\t= {wout}")

dloss/dprob  	= tensor([-1.2053], grad_fn=<MulBackward0>)
dprob/dscore 	= 0.23946520686149597
dscore/dhact 	= tensor([-1.3355,  1.2879,  0.0274], requires_grad=True)
dhact/dh 	= tensor([1., 1., 1.])
dh/dW =
tensor([[1., 1., 1.],
        [2., 2., 2.]])

tensor([[ 0.3855, -0.3717, -0.0079],
        [ 0.7709, -0.7435, -0.0158]], grad_fn=<MulBackward0>)


In [199]:
# compare our manually computed derivatives against those calculated by pytorch

# zero gradients and re-do forward pass
W.grad = None
wout.grad = None

h = x@W
hact = relu(h)
score = hact@wout
prob = sigmoid(score)
loss = (prob - t)**2

loss.backward()

print("dW")
print(f"pytorch grad: \t{wout.grad}\nour grad: \t{dloss_dwout}")
print(f"are they equal? {wout.grad == dloss_dwout}")
print()
print(f"pytorch grad: \n{W.grad}\n\nour grad: \n{dloss_dW}\n")
print(f"are they equal?\n{W.grad == dloss_dW}")

pytorch grad: 	tensor([-0.5941, -0.5121, -0.4969])
our grad: 	tensor([-0.5941, -0.5121, -0.4969], grad_fn=<MulBackward0>)
are they equal? tensor([True, True, True])

pytorch grad: 
tensor([[ 0.3855, -0.3717, -0.0079],
        [ 0.7709, -0.7435, -0.0158]])

our grad: 
tensor([[ 0.3855, -0.3717, -0.0079],
        [ 0.7709, -0.7435, -0.0158]], grad_fn=<MulBackward0>)

are they equal?
tensor([[True, True, True],
        [True, True, True]])


=> WE GET THE SAME GRADIENTS AS PYTORCH